In [1]:
!pip install pandas
!pip install numpy
!pip install scikit-learn


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [3]:
import requests

file_path = "data/admission.csv"

def download_database():
  response = requests.get("https://raw.githubusercontent.com/lucaslattari/MachineLearningSeries/refs/heads/master/Ep%202/admission.csv", stream=True)
  response.raise_for_status()

  with open(file_path, "wb") as file:
    for chunk in response.iter_content(chunk_size=8192):
      file.write(chunk)

In [4]:
download_database()

In [ ]:
import pandas as pd
import numpy as np

def extract_data():
  df = pd.read_csv(file_path, delimiter=";")
  x = df.iloc[:,:-1].values
  y = df.iloc[:,-1].values
  return x, y

extract_data()

(array([['Lucas', 337, 118, 4, 4.5, 4.5, 9.65, 1],
        ['Ana', 324, 107, 4, 4.0, 4.5, 8.87, 1],
        ['Jose', 316, 104, 3, 3.0, 3.5, 8.0, 1],
        ['Carlos', 322, 110, 3, 3.5, 2.5, 8.67, 1],
        ['Zileide', 314, 103, 2, 2.0, 3.0, 8.21, 0],
        ['Joana', 330, 115, 5, 4.5, 3.0, 9.34, 1],
        ['Davi', 321, 109, 3, 3.0, 4.0, 8.2, 1],
        ['Daniel', 308, 101, 2, 3.0, 4.0, 7.9, 0],
        ['Marcelo', 302, 102, 1, 2.0, 1.5, 8.0, 0]], dtype=object),
 array([1, 1, 1, 1, 0, 1, 1, 0, 0]))

In [26]:
from sklearn.impute import SimpleImputer

def impute_data():
  X, y = extract_data()
  imputer = SimpleImputer(missing_values=np.nan, strategy="median")
  imputer = imputer.fit_transform(X[:,1:])
  X[:,1:] = imputer
  return X, y

In [33]:
from sklearn.preprocessing import LabelEncoder
from pprint import pprint


# Decodifica as labels para IDs, ao treinar o modelo não é recomendado passar os dados com os ids
def encoder_label_data():
  labelencoder_x = LabelEncoder()
  X, y = extract_data()
  X[:, 0] = labelencoder_x.fit_transform(X[:, 0])
  pprint(X)


# RECOMENDAÇÃO - Usar One-Hot Encoding para decodificar as label em binários
def one_hot_encoding():
  X, y = impute_data()
  names_column = X[:, 0]
  names_one_hot = pd.get_dummies(names_column, dtype=int)
  numerical_columns = X[:, 1:]
  X = np.insert(numerical_columns, 0, names_one_hot.values, axis=1)
  return X, y


In [34]:
encoder_label_data()

array([[6, 337, 118, 4, 4.5, 4.5, 9.65, 1],
       [0, 324, 107, 4, 4.0, 4.5, 8.87, 1],
       [5, 316, 104, 3, 3.0, 3.5, 8.0, 1],
       [1, 322, 110, 3, 3.5, 2.5, 8.67, 1],
       [8, 314, 103, 2, 2.0, 3.0, 8.21, 0],
       [4, 330, 115, 5, 4.5, 3.0, 9.34, 1],
       [3, 321, 109, 3, 3.0, 4.0, 8.2, 1],
       [2, 308, 101, 2, 3.0, 4.0, 7.9, 0],
       [7, 302, 102, 1, 2.0, 1.5, 8.0, 0]], dtype=object)


In [35]:
one_hot_encoding()

(array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 337.0, 118.0, 4.0, 4.5, 4.5, 9.65,
         1.0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 324.0, 107.0, 4.0, 4.0, 4.5, 8.87,
         1.0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 316.0, 104.0, 3.0, 3.0, 3.5, 8.0, 1.0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 322.0, 110.0, 3.0, 3.5, 2.5, 8.67,
         1.0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 314.0, 103.0, 2.0, 2.0, 3.0, 8.21,
         0.0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 330.0, 115.0, 5.0, 4.5, 3.0, 9.34,
         1.0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 321.0, 109.0, 3.0, 3.0, 4.0, 8.2, 1.0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 308.0, 101.0, 2.0, 3.0, 4.0, 7.9, 0.0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 302.0, 102.0, 1.0, 2.0, 1.5, 8.0, 0.0]],
       dtype=object),
 array([1, 1, 1, 1, 0, 1, 1, 0, 0]))

In [38]:
from sklearn.model_selection import train_test_split

# Separação de dados para treino e dados para teste
def split_data():
  X, y = one_hot_encoding()
  XTrain, XTest, YTrain, YTest = train_test_split(X, y, test_size=0.2)
  print("XTrain:", XTrain)
  print("XTest:", XTest)

  # return XTrain, XTest, YTrain, YTest
split_data()

XTrain: [[0 0 0 0 1 0 0 0 0 302.0 102.0 1.0 2.0 1.5 8.0 0.0]
 [0 0 0 0 0 0 1 0 0 322.0 110.0 3.0 3.5 2.5 8.67 1.0]
 [0 0 0 0 0 0 0 0 1 308.0 101.0 2.0 3.0 4.0 7.9 0.0]
 [0 0 0 0 0 0 0 1 0 316.0 104.0 3.0 3.0 3.5 8.0 1.0]
 [0 0 0 1 0 0 0 0 0 324.0 107.0 4.0 4.0 4.5 8.87 1.0]
 [0 0 0 0 0 1 0 0 0 314.0 103.0 2.0 2.0 3.0 8.21 0.0]
 [0 1 0 0 0 0 0 0 0 337.0 118.0 4.0 4.5 4.5 9.65 1.0]]
XTest: [[0 0 1 0 0 0 0 0 0 330.0 115.0 5.0 4.5 3.0 9.34 1.0]
 [1 0 0 0 0 0 0 0 0 321.0 109.0 3.0 3.0 4.0 8.2 1.0]]
